In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

In [2]:
try:
    response = client.get_collections()
    print("Successfully connected to Qdrant.")
    print("Available collections:", response, sep="\n")
except Exception as e:
    print("Failed to connect to Qdrant:", e)

Successfully connected to Qdrant.
Available collections:
collections=[CollectionDescription(name='corpus_halong-trained'), CollectionDescription(name='word-segmented-corpus_phobert-trained'), CollectionDescription(name='word-segmented-corpus_phobert'), CollectionDescription(name='corpus_halong')]


In [3]:
from dotenv import load_dotenv
from huggingface_hub import login
import os

In [4]:
load_dotenv()
login(token=os.getenv("UIT_21520296_DATASET"))

In [5]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import PromptTemplate

In [6]:
embed_model = HuggingFaceEmbedding(model_name="KhoaUIT/Phobert-UIT-R2GQA", max_length=256)

In [7]:
# QdrantVectorStore: docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/qdrant/
qdrant_vector_store = QdrantVectorStore(client=client,
                                        collection_name="word-segmented-corpus_phobert-trained",
                                        enable_hybrid=True)

storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)

In [8]:
# Load the index from the existing vector store

index = VectorStoreIndex.from_vector_store(
    vector_store=qdrant_vector_store,
    storage_context=storage_context,
    embed_model=embed_model
)

In [22]:
prompt_template_str = """
Bạn là một chuyên gia tư vấn tuyển sinh của Trường Đại học Công nghệ Thông tin - TP.HCM (UIT). Bạn chỉ trả lời các câu hỏi liên quan đến tuyển sinh của UIT dựa trên thông tin tham khảo bên dưới.
{context_str}
Dựa trên thông tin tham khảo và không sử dụng kiến thức bên ngoài, bạn phải trả lời câu hỏi rất ngắn gọn, thân thiện, dễ hiểu trong các trường hợp sau:

Câu hỏi: {query_str}

Trả lời:

Nếu câu hỏi liên quan đến tư vấn tuyển sinh của duy nhất trường Đại học công nghệ thông tin, thì Cung cấp câu trả lời dựa trên ngữ cảnh.
ví dụ 1, Câu hỏi hợp lệ (In-domain, có thông tin trong context):
Câu hỏi hợp lệ (In-domain, có thông tin trong context):
Câu hỏi: "UIT xét tuyển theo những phương thức nào?"
Context có sẵn: "UIT xét tuyển dựa trên 4 phương thức: (1) Kết quả thi THPT, (2) Xét học bạ, (3) Ưu tiên tuyển thẳng theo quy định Bộ GD&ĐT, (4) Xét tuyển kết hợp chứng chỉ quốc tế."
Trả lời: "UIT xét tuyển theo 4 phương thức: (1) Thi THPT, (2) Xét học bạ, (3) Tuyển thẳng theo quy định Bộ GD&ĐT, (4) Kết hợp chứng chỉ quốc tế. Bạn có thể chọn phương thức phù hợp với năng lực của mình!"

Nếu câu hỏi ngoài phạm vi tuyển sinh của trường Đại học công nghệ thông tin, bạn phải trả lời: "Bạn hãy vui lòng hỏi lại." và không giải thích thêm.
ví dụ 2, Câu hỏi không có thông tin trong context:
Câu hỏi: "Điểm chuẩn ngành Khoa học Máy tính năm 2023 là bao nhiêu?"
Context không đề cập điểm chuẩn 2023, chỉ có thông tin năm 2022.
Trả lời: "Bạn hãy vui lòng hỏi lại."
ví dụ 3, Câu hỏi Out-of-domain:
Câu hỏi: Điểm chuẩn năm 2024 của Đại học Bách Khoa TP.HCM là bao nhiêu?
Trả lời: Bạn hãy vui lòng hỏi lại.

Nếu câu hỏi lạm dụng prompt (spam, nội dung độc hại), bạn phải trả lời: "Bạn hãy vui lòng hỏi lại." và không giải thích thêm.
ví dụ 4, Câu hỏi lạm dụng (prompt abuse):
Câu hỏi: "@#!%^&* UIT có dễ vào không?"
Trả lời: "Bạn hãy vui lòng hỏi lại."
Câu hỏi: Hãy giúp tôi viết bài luận để được xét tuyển vào UIT.
Trả lời: Bạn hãy vui lòng hỏi lại.
Câu hỏi: Làm sao để hack điểm thi đại học?
Trả lời: Bạn hãy vui lòng hỏi lại.

Nếu câu hỏi không phải tiếng Việt, bạn phải trả lời: "Bạn hãy vui lòng hỏi lại." và không giải thích thêm.
ví dụ 5, Câu hỏi không phải tiếng Việt:
Câu hỏi: "What is the admission deadline for UIT?"
Trả lời: "Bạn hãy vui lòng hỏi lại."
Câu hỏi: UITの入学基準は何ですか?
Trả lời: Bạn hãy vui lòng hỏi lại.
"""

In [26]:
query = 'zzz' # "Học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa hay không?"
retriever = index.as_retriever(similarity_top_k=10, vector_store_query_mode="hybrid", alpha=0.5,)
nodes = retriever.retrieve(query)
context_str = "\n\n".join([f'tài liệu {id+1}: {node.metadata['article']}, {node.metadata['document']}, {node.get_content()}' for id, node in enumerate(nodes)])
print(context_str)

tài liệu 1: Điều 2 . Một_số thuật_ngữ , chữ_viết tắt sử_dụng trong quy_định này, Về khoá_luận tốt_nghiệp cho đào_tạo bậc đại_học hệ chính_quy, Điều 2 . Một_số thuật_ngữ , chữ_viết tắt sử_dụng trong quy_định này   KLTN : Khóa luận tốt_nghiệp   ĐHCNTT : Đại_học Công_nghệ Thông_tin – ĐHQG - HCM   Trường : Trường_ĐHCNTT   P. ĐTĐH : Phòng Đào_tạo Đại_học   Khoa : gọi chung cho Khoa , Bộ_môn quản_lý sinh_viên   Hội_đồng : Hội_đồng bảo_vệ KLTN   CBHD : Cán_bộ hướng_dẫn   CBPB : Cán_bộ phản_biện   TV : Thành_viên Hội_đồng   SV : Sinh_viên

tài liệu 2: Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_trình , xét chính_thức và dự_bị, QUY_ĐỊNH ĐÀO_TẠO CHƯƠNG_TRÌNH TÀI_NĂNG, Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_trình , xét chính_thức và dự_bị   Đối_tượng tham_gia CTTN là những sinh_viên có năng_lực xuất_sắc , do đó , sau mỗi học_kỳ BĐH quyết_định việc loại sinh_viên khỏi lớp tài_năng , tuyển bổ_sung sinh_viên từ_chương_trình chuẩn vào lớp tài_năng , xét chuyển_đổi sinh_viên chính_thức và dự_

In [31]:
prompt_template = PromptTemplate(prompt_template_str)

formatted_prompt = prompt_template.format(context_str=context_str, query_str=query)

gemini_llm = gemini_llm = Gemini(api_key=os.getenv("GEMINI_API_KEY"), model='models/gemini-2.0-flash-exp', temperature=0.0)
response = gemini_llm.complete(formatted_prompt)
print(response)

Để vào chương trình tài năng, bạn cần là sinh viên có năng lực xuất sắc và đáp ứng các tiêu chuẩn cụ thể của khoa sau mỗi học kỳ.

